In [1]:
import pandas as pd
import os
import sys
import tensorflow as tf
import numpy as np
import tensorflow_hub as hub

In [2]:
import wandb
from wandb.keras import WandbCallback

In [2]:
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [3]:
feature_names = ["product", "sub_product", "issue", "sub_issue", "state", "zip_code", "company", "company_response", "timely_response", "consumer_disputed", "consumer_complaint_narrative"]
one_hot_features = ['product', 'sub_product', 'company_response', 'state', 'issue']
numeric_features = ['zip_code']
text_features = ['consumer_complaint_narrative']

In [5]:
df = pd.read_csv('../data/consumer_complaints_with_narrative.csv', usecols=feature_names)

In [6]:
df.head()

,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company,state,zip_code,company_response,timely_response,consumer_disputed
0,Debt collection,I do not know,Disclosure verification of debt,Right to dispute notice not received,I was denied employment because of a judgment ...,Encore Capital Group,NY,113XX,Closed with explanation,Yes,0
1,Credit reporting,NaN,Improper use of my credit report,Report improperly shared by CRC,I have a credit card through XXXX XXXX and XXX...,Experian,IL,606XX,Closed with non-monetary relief,Yes,0
2,Debt collection,I do not know,Cont'd attempts collect debt not owed,Debt is not mine,Almost daily phone calls from Stellar Recovery...,Stellar Recovery Inc.,MI,480XX,Closed with explanation,Yes,1
3,Mortgage,Conventional fixed mortgage,"Loan servicing, payments, escrow account",NaN,I submitted my monthly mortgage payment to Pri...,Primary Residential Mortgage,CT,066XX,Closed with monetary relief,Yes,0
4,Student loan,Non-federal student loan,Dealing with my lender or servicer,Received bad information about my loan,I contacted America Education Services in XX/X...,AES/PHEAA,FL,321XX,Closed with explanation,Yes,1


In [7]:
for col in one_hot_features:
    print(col)
    print(df[col].nunique())

product
11
sub_product
45
company_response
5
state
60
issue
90


In [8]:
df['consumer_disputed'] = df['consumer_disputed'].map({'Yes':1, 'No':0})

In [9]:
for feature in one_hot_features:
    df[feature] = df[feature].astype("category").cat.codes

In [10]:
one_hot_x = [pd.np.asarray(tf.keras.utils.to_categorical(df[feature_name].values)) for feature_name in one_hot_features]

/var/folders/7h/_rbt3v8d3vd1h3c8f1zdrsmm0000gn/T/ipykernel_43478/1472151705.py:1: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  one_hot_x = [pd.np.asarray(tf.keras.utils.to_categorical(df[feature_name].values)) for feature_name in one_hot_features]


In [11]:
embedding_x = [pd.np.asarray(df[feature_name].values).reshape(-1) for feature_name in text_features]

/var/folders/7h/_rbt3v8d3vd1h3c8f1zdrsmm0000gn/T/ipykernel_43478/1459859265.py:1: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  embedding_x = [pd.np.asarray(df[feature_name].values).reshape(-1) for feature_name in text_features]


In [12]:
df['zip_code'] = df['zip_code'].str.replace('X', '0', regex=True)

In [13]:
df['zip_code'] = df['zip_code'].str.replace(r'\[|\*|\+|\-|`|\.|\ |\$|\/|!|\(', '0', regex=True)

In [14]:
df['zip_code'] = df['zip_code'].fillna(0)

In [15]:
df['zip_code'] = df['zip_code'].astype('int32')

In [16]:
df['zip_code'] = df['zip_code'].apply(lambda x: x//10000)

In [17]:
numeric_x = [df['zip_code'].values]

In [18]:
X = one_hot_x + numeric_x + embedding_x

In [19]:
y = np.asarray(df["consumer_disputed"], dtype=np.uint8).reshape(-1)

In [22]:
def get_model(show_summary=True):
    """
    Function defines a Keras model and returns the model as Keras object
    """
    wandb.init(project="consumer-complaints")
    config = wandb.config
    config.name='final_features_wide'
    config.hidden_layer_size = 256
    config.optimizer = 'adam'
    config.learning_rate = 0.001
    config.data_version = 'cc_imbalanced_narrative'
    config.one_hot_features = one_hot_features
    config.numeric_features = numeric_features
    config.text_features = text_features
    
    # one-hot categorical features
    num_products = 11
    num_sub_products = 45
    num_company_responses = 5
    num_states = 60
    num_issues = 90

    input_product = tf.keras.Input(shape=(num_products,), name="product_xf")
    input_sub_product = tf.keras.Input(shape=(num_sub_products,), name="sub_product_xf")
    input_company_response = tf.keras.Input(shape=(num_company_responses,), name="company_response_xf")
    input_state = tf.keras.Input(shape=(num_states,), name="state_xf")
    input_issue = tf.keras.Input(shape=(num_issues,), name="issue_xf")
    
    # numeric features
    input_zip_code = tf.keras.Input(shape=(1,), name="zip_code_xf")

    # text features
    input_narrative = tf.keras.Input(shape=(1,), name="narrative_xf", dtype=tf.string)

    # embed text features
    module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
    embed = hub.KerasLayer(module_url)
    reshaped_narrative = tf.reshape(input_narrative, [-1])
    embed_narrative = embed(reshaped_narrative) 
    deep_ff = tf.keras.layers.Reshape((512, ), input_shape=(1, 512))(embed_narrative)
    
    deep = tf.keras.layers.Dense(256, activation='relu')(deep_ff)
    deep = tf.keras.layers.Dense(64, activation='relu')(deep)
    deep = tf.keras.layers.Dense(16, activation='relu')(deep)

    wide_ff = tf.keras.layers.concatenate(
        [input_product, input_sub_product, input_company_response, 
         input_state, input_issue, input_zip_code])
    wide = tf.keras.layers.Dense(16, activation='relu')(wide_ff)


    both = tf.keras.layers.concatenate([deep, wide])

    output = tf.keras.layers.Dense(1, activation='sigmoid')(both) 

    _inputs = [input_product, input_sub_product, input_company_response,  
               input_state, input_issue, input_zip_code, input_narrative]

    keras_model = tf.keras.models.Model(_inputs, output)
    keras_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                     loss='binary_crossentropy',  
                     metrics=[
                         tf.keras.metrics.BinaryAccuracy(),
                         tf.keras.metrics.TruePositives()
                         ])
    if show_summary:
        keras_model.summary()

    return keras_model

In [23]:
model = get_model(show_summary=False)

2021-11-23 12:45:04.267354: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-23 12:45:08.180375: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [24]:
model.fit(x=X, 
          y=y, 
          batch_size=32, 
          validation_split=0.2, 
          epochs=5, 
          callbacks=[WandbCallback()]
)

Epoch 1/5
1670/1670 [==============================] - 56s 32ms/step - loss: 0.0062 - binary_accuracy: 0.9973 - true_positives: 0.0000e+00 - val_loss: 1.2860e-06 - val_binary_accuracy: 1.0000 - val_true_positives: 0.0000e+00
Epoch 2/5
1670/1670 [==============================] - 52s 31ms/step - loss: 4.0018e-07 - binary_accuracy: 1.0000 - true_positives: 0.0000e+00 - val_loss: 3.3683e-07 - val_binary_accuracy: 1.0000 - val_true_positives: 0.0000e+00
Epoch 3/5
1670/1670 [==============================] - 52s 31ms/step - loss: 1.0129e-07 - binary_accuracy: 1.0000 - true_positives: 0.0000e+00 - val_loss: 1.2363e-07 - val_binary_accuracy: 1.0000 - val_true_positives: 0.0000e+00
Epoch 4/5
1670/1670 [==============================] - 51s 31ms/step - loss: 3.3623e-08 - binary_accuracy: 1.0000 - true_positives: 0.0000e+00 - val_loss: 5.1761e-08 - val_binary_accuracy: 1.0000 - val_true_positives: 0.0000e+00
Epoch 5/5
1670/1670 [==============================] - 52s 31ms/step - loss: 1.2559e-08 

In [7]:
#from IPython.display import Image

file_name = 'model.png'
tf.keras.utils.plot_model(model, to_file=file_name)
#Image(filename=file_name)

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


In [ ]:
def get_model(show_summary=True):
    """
    Function defines a Keras model and returns the model as Keras object
    """
    
    # one-hot categorical features
    num_products = 11
    num_sub_products = 45
    num_company_responses = 5
    num_states = 60
    num_issues = 90

    input_product = tf.keras.Input(shape=(num_products,), name="product_xf")
    input_sub_product = tf.keras.Input(shape=(num_sub_products,), name="sub_product_xf")
    input_company_response = tf.keras.Input(shape=(num_company_responses,), name="company_response_xf")
    input_state = tf.keras.Input(shape=(num_states,), name="state_xf")
    input_issue = tf.keras.Input(shape=(num_issues,), name="issue_xf")
    
    # numeric features
    input_zip_code = tf.keras.Input(shape=(1,), name="zip_code_xf")

    # text features
    input_narrative = tf.keras.Input(shape=(1,), name="narrative_xf", dtype=tf.string)

    # embed text features
    module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
    embed = hub.KerasLayer(module_url)
    reshaped_narrative = tf.reshape(input_narrative, [-1])
    embed_narrative = embed(reshaped_narrative) 
    deep_ff = tf.keras.layers.Reshape((512, ), input_shape=(1, 512))(embed_narrative)
    
    deep = tf.keras.layers.Dense(256, activation='relu')(deep_ff)
    deep = tf.keras.layers.Dense(64, activation='relu')(deep)
    deep = tf.keras.layers.Dense(16, activation='relu')(deep)

    wide_ff = tf.keras.layers.concatenate(
        [input_product, input_sub_product, input_company_response, 
         input_state, input_issue, input_zip_code])
    wide = tf.keras.layers.Dense(16, activation='relu')(wide_ff)


    both = tf.keras.layers.concatenate([deep, wide])

    output = tf.keras.layers.Dense(1, activation='sigmoid')(both) 

    _inputs = [input_product, input_sub_product, input_company_response,  
               input_state, input_issue, input_zip_code, input_narrative]

    keras_model = tf.keras.models.Model(_inputs, output)
    keras_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                     loss='binary_crossentropy',  
                     metrics=[
                         tf.keras.metrics.BinaryAccuracy(),
                         tf.keras.metrics.TruePositives()
                         ])
    if show_summary:
        keras_model.summary()

    return keras_model